In [ ]:
from BrickPi import *
BrickPiSetup()

BrickPi.MotorEnable[PORT_A] = 1
BrickPi.MotorEnable[PORT_D] = 1		

BrickPi.SensorType[PORT_4] = TYPE_SENSOR_RAW

BrickPiSetupSensors()

In [ ]:
S = [760,680,445,350,297,266,248,230]
D = [3, 4, 8, 12, 16, 20, 24, 38]

def interpolateDistance(V):
    i = 0
    while i < len(S) and S[i] > V:
        i = i + 1
    if i==0:
        d = float(D[0])
    elif i==len(S):
        d = float(D[-1])
    else:
        SI = S[i-1]
        SF = S[i]
        DI = D[i-1]
        DF = D[i]
        d = DI + float(DF-DI)*(V-SI)/(SF-SI)
    return d

In [ ]:
from IPython.display import clear_output
errors = []
try:
    while True:
        clear_output()
        V = BrickPi.Sensor[PORT_4]
        if V is None:
            print("ERROR: V is None")
        elif V > 1000 or V < 100:
            print("ERROR: V = %d" % V)
            errors.append(V)
            BrickPiSetupSensors()
            BrickPi.MotorSpeed[PORT_A] = 0
            BrickPi.MotorSpeed[PORT_D] = 0
        else:
            d = interpolateDistance(V)
            print(V,d)
            if d>=16 and d<19:
                BrickPi.MotorSpeed[PORT_A] = 60
                BrickPi.MotorSpeed[PORT_D] = 60
            elif d<16:
                BrickPi.MotorSpeed[PORT_A] =  0
                BrickPi.MotorSpeed[PORT_D] = 60
            else:
                BrickPi.MotorSpeed[PORT_A] =  60
                BrickPi.MotorSpeed[PORT_D] =  40
                
        time.sleep(0.01)
        BrickPiUpdateValues()

except KeyboardInterrupt:
        print("Errors: %d" % len(errors))
        print(errors)